<a href="https://colab.research.google.com/github/thollaar/BUAN5405-docs/blob/master/Cadinality_reducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cardinality Reducer

This code produces a cardinality reducer for your __categorical__ data.

In [ ]:
#import sklearn.datasets
import pandas as pd
import numpy as np


In [ ]:
data_url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
order_df = pd.read_csv(data_url, sep = '\t')

In [ ]:
order_df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


Clearly ``item_name`` is a __categorical__ field. But we still need to make sure.

In [ ]:
order_df.dtypes

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

We still want to convert the data to the ``category`` data type to make sure.

In [ ]:
item_names = order_df['item_name'].astype('category')
item_names.dtype

CategoricalDtype(categories=['6 Pack Soft Drink', 'Barbacoa Bowl', 'Barbacoa Burrito',
                  'Barbacoa Crispy Tacos', 'Barbacoa Salad Bowl',
                  'Barbacoa Soft Tacos', 'Bottled Water', 'Bowl', 'Burrito',
                  'Canned Soda', 'Canned Soft Drink', 'Carnitas Bowl',
                  'Carnitas Burrito', 'Carnitas Crispy Tacos',
                  'Carnitas Salad', 'Carnitas Salad Bowl',
                  'Carnitas Soft Tacos', 'Chicken Bowl', 'Chicken Burrito',
                  'Chicken Crispy Tacos', 'Chicken Salad',
                  'Chicken Salad Bowl', 'Chicken Soft Tacos', 'Chips',
                  'Chips and Fresh Tomato Salsa', 'Chips and Guacamole',
                  'Chips and Mild Fresh Tomato Salsa',
                  'Chips and Roasted Chili Corn Salsa',
                  'Chips and Roasted Chili-Corn Salsa',
                  'Chips and Tomatillo Green Chili Salsa',
                  'Chips and Tomatillo Red Chili Salsa',
               

In this case we converted the data ourselves. For other data, we should test if the column(s) we want to work on are indeed categorical (``category`` or ``object``).

In [ ]:
if order_df['item_name'].dtypes == 'O':
  print('categorical!')
  # do whatever you want
else:
  raise(TypeError('Not categorical data!'))

categorical!


We can use two types of methods to reduce the cardinality:
- Top-N: preserve the top-N values by __count__, and replace the other values as 'other'
- Threshold: preserve the values with __frequencies__ higher than the threshold, and replace the other values as 'other'
- Other methods: such as clustering based method, binning based method, ...

Let's look at the top-N method first ($N==5$).

In [ ]:
order_df['item_name'].value_counts() # automatically ranked

Chicken Bowl                             726
Chicken Burrito                          553
Chips and Guacamole                      479
Steak Burrito                            368
Canned Soft Drink                        301
Steak Bowl                               211
Chips                                    211
Bottled Water                            162
Chicken Soft Tacos                       115
Chicken Salad Bowl                       110
Chips and Fresh Tomato Salsa             110
Canned Soda                              104
Side of Chips                            101
Veggie Burrito                            95
Barbacoa Burrito                          91
Veggie Bowl                               85
Carnitas Bowl                             68
Barbacoa Bowl                             66
Carnitas Burrito                          59
Steak Soft Tacos                          55
6 Pack Soft Drink                         54
Chips and Tomatillo Red Chili Salsa       48
Chicken Cr

In [ ]:
# Let's assume we want top-5 values.
N = 5

topN = order_df['item_name'].value_counts()[:N].index
topN

Index(['Chicken Bowl', 'Chicken Burrito', 'Chips and Guacamole',
       'Steak Burrito', 'Canned Soft Drink'],
      dtype='object')

In [ ]:
# reduced item names

item_names_rdced = pd.Series(np.where(order_df['item_name'].isin(topN), order_df['item_name'], 'other'))

item_names_rdced[:10]

0            other
1            other
2            other
3            other
4     Chicken Bowl
5     Chicken Bowl
6            other
7    Steak Burrito
8            other
9    Steak Burrito
dtype: object

In [ ]:
item_names_rdced.nunique()

6

Now let's check the threshold based methods ($thres >= 0.05$). 

In [ ]:
cat_freqs = order_df['item_name'].value_counts(normalize=True)
cat_freqs

Chicken Bowl                             0.157075
Chicken Burrito                          0.119645
Chips and Guacamole                      0.103635
Steak Burrito                            0.079619
Canned Soft Drink                        0.065123
Steak Bowl                               0.045651
Chips                                    0.045651
Bottled Water                            0.035050
Chicken Soft Tacos                       0.024881
Chicken Salad Bowl                       0.023799
Chips and Fresh Tomato Salsa             0.023799
Canned Soda                              0.022501
Side of Chips                            0.021852
Veggie Burrito                           0.020554
Barbacoa Burrito                         0.019688
Veggie Bowl                              0.018390
Carnitas Bowl                            0.014712
Barbacoa Bowl                            0.014280
Carnitas Burrito                         0.012765
Steak Soft Tacos                         0.011900


In [ ]:
thres = 0.05

top_cat = cat_freqs[cat_freqs>thres].index
top_cat

Index(['Chicken Bowl', 'Chicken Burrito', 'Chips and Guacamole',
       'Steak Burrito', 'Canned Soft Drink'],
      dtype='object')

In [ ]:
# reduced item names

item_names_rdced_freq = np.where(order_df['item_name'].isin(top_cat), order_df['item_name'], 'other')

item_names_rdced_freq[:10]

array(['other', 'other', 'other', 'other', 'Chicken Bowl', 'Chicken Bowl',
       'other', 'Steak Burrito', 'other', 'Steak Burrito'], dtype=object)

Now it's time to put everything together.

In [ ]:
order_df.dtypes

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

In [ ]:
def card_reducer(cat_col, reduce_type='topn', N=3, thres=0.01, rpl_value = 'other'):
  """
  This function reduces the cardinality of a categorcal feature, or multiple features.
  Only the frequent values in the feature will be preserved, the other values will be changed to 'other'.

  ARGUMENTS
  -------------
  - cat_col: (pandas Series) feature to be reduced, must be of categorical type
  - reduce_type: (string) reduce method, can be 'topn' or 'thres'. 
    - 'topn' refers to the Top-N method, 'thres' refers to the threshold-based method. Default is 'topn'.
  - N: (int) number of top count values to preserve, only used if 'topn' is selected in `type`.
  - thres: (float) threshold of the frequency. Values with higher than the thresold are preserved. Only used if 'thres' is selected in type.
  - rpl_value: (string) string used to replace the less frequent values. Default is 'other'.

  OUTPUT
  --------------
  - reduced_values: (pandas Series) reduced column(s), with the names as 'col_name + _reduced'. And col_name is the original column name.
  """
  # test for the data type
  if cat_col.dtypes == 'O':
    pass
  
  else:
    #raise(TypeError('The feature is NOT categorical!'))


    # Top-N Method
    #### top_cat contains the preserved values
    if (reduce_type == 'topn') & isinstance(N, int): # test if N is integer
      top_cat = cat_col.value_counts()[:N].index
    
    # Threshold Method
    #### top_cat contains the preserved values
    elif (reduce_type == 'thres') & isinstance(thres, float): # test if thres is float
      cat_freqs = cat_col.value_counts(normalize=True)
      top_cat = cat_freqs[cat_freqs>thres].index
    else:
      raise(TypeError('N has to be integer! OR thres has to be float!'))

    new_name = cat_col.name + '_reduced'
    # reduced values
    if isinstance(rpl_value, str): # test if the rpl_value is string
      reduced_values = pd.Series(np.where(cat_col.isin(top_cat), cat_col, rpl_value), name=new_name)
    else:
      raise(TypeError('replace value has to be string!'))

    ###############################################################################
    # Build-in tests
    #### reduced feature should be of the same shape compared to the original
    if reduced_values.shape == cat_col.shape: 
      pass
    else:
      raise(ValueError('Shape of the reduced feature is different from the original.'))

    #### test for data type of reduced feature
    if reduced_values.dtypes == 'O':
      pass
    # do whatever you want
    else:
      raise(TypeError('The reduced feature is NOT categorical!'))


    #### test for the top-N method
    if (reduce_type == 'topn') & (reduced_values.nunique() == N + 1):
      pass

    #### test for the threshold method
    elif (reduce_type == 'thres') & ((reduced_values.value_counts(normalize=True) >= thres).all()):
      pass
    else:
      raise(ValueError('Not only the Top-N values are preserved, or values with the frequency lower than the threshold are preserved.'))
    ###############################################################################
    
    return reduced_values

In [ ]:
# tony test dataframe we have now
order_df.head(10)
order_df['item_name'] = order_df['item_name'].astype('category')
order_df.dtypes

order_id                 int64
quantity                 int64
item_name             category
choice_description      object
item_price              object
dtype: object

In [ ]:
order_df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [ ]:
order_df['item_name'] = card_reducer(order_df['item_name'])

In [ ]:
# tony test if card_reducer function worked
order_df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,other,NaN,$2.39
1,1,1,other,[Clementine],$3.39
2,1,1,other,[Apple],$3.39
3,1,1,other,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [35]:
order_df['item_name'].value_counts()

other                  2864
Chicken Bowl            726
Chicken Burrito         553
Chips and Guacamole     479
Name: item_name, dtype: int64